# Proper big notebook

In [23]:
#import packages
import pandas as pd
import tensorflow as tf
import numpy as np
import time
import os
import zipfile
import platform
from scipy.signal import medfilt
from tqdm import tqdm
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc, accuracy_score, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils


import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10, 6)
mpl.rcParams['axes.grid'] = True
mpl.rcParams['legend.fontsize'] = 'large'

In [6]:
if platform.system() == "Darwin":
    root_directory = '/Users/lorenzobarbiero/Documents/GitHub/CNN-Gait-recognition'
else:
    root_directory = None ##Francesco qui metti il tuo path
    
print('root directory is ' + root_directory)

root directory is /Users/lorenzobarbiero/Documents/GitHub/CNN-Gait-recognition


## Train/Test Split

Load reference file, separate the two different data types and one hot encode the labels

In [160]:
# Create a dataframe containing all the files and the labels from the reference file
reference_df = pd.read_csv(root_directory+'/reference_document.csv')
reference_df


,File Name,Path from Root,Type,Label
0,Sbj2_steppage_T9_P.csv,subject2/steppage/trial9/,pressure,steppage
1,Sbj2_steppage_T9_S.csv,subject2/steppage/trial9/,skeleton,steppage
2,Sbj2_steppage_T14_P.csv,subject2/steppage/trial14/,pressure,steppage
3,Sbj2_steppage_T14_S.csv,subject2/steppage/trial14/,skeleton,steppage
4,Sbj2_steppage_T13_S.csv,subject2/steppage/trial13/,skeleton,steppage
...,...,...,...,...
2875,Sbj9_trendelenburg_T11_S.csv,subject9/trendelenburg/trial11/,skeleton,trendelenburg
2876,Sbj9_trendelenburg_T18_S.csv,subject9/trendelenburg/trial18/,skeleton,trendelenburg
2877,Sbj9_trendelenburg_T18_P.csv,subject9/trendelenburg/trial18/,pressure,trendelenburg
2878,Sbj9_trendelenburg_T20_S.csv,subject9/trendelenburg/trial20/,skeleton,trendelenburg


In [65]:
# Create a dataframe containing all the files and the labels from the reference file
reference_df = pd.read_csv(root_directory+'/reference_document.csv')

#split the dataframe into skeleton and pressure, drop the type column
reference_df_s = reference_df[reference_df['Type']=='skeleton'].drop(columns='Type')
reference_df_p = reference_df[reference_df['Type']=='pressure'].drop(columns='Type')

#train_test split
train_s, val_s = train_test_split(reference_df_s, test_size=0.2, stratify=reference_df_s['Label'], random_state=123)
train_p, val_p = train_test_split(reference_df_p, test_size=0.2, stratify=reference_df_p['Label'], random_state=123)

#frequence counts
print("Train Skelly")
print(train_s['Label'].value_counts())
print("Val Skelly")
print(val_s['Label'].value_counts())
print("Train Pressure")
print(train_p['Label'].value_counts())
print("Val Pressure")
print(val_p['Label'].value_counts())

Train Skelly
trendelenburg    192
steppage         192
stifflegged      192
antalgic         192
lurching         192
normal           192
Name: Label, dtype: int64
Val Skelly
steppage         48
normal           48
lurching         48
stifflegged      48
trendelenburg    48
antalgic         48
Name: Label, dtype: int64
Train Pressure
trendelenburg    192
steppage         192
stifflegged      192
antalgic         192
lurching         192
normal           192
Name: Label, dtype: int64
Val Pressure
steppage         48
normal           48
lurching         48
stifflegged      48
trendelenburg    48
antalgic         48
Name: Label, dtype: int64


1 hot encode labels

In [59]:
#1 hoe labels
#create encoder
#encoder = LabelEncoder()

#train_s_labs = np_utils.to_categorical(encoder.fit(train_s['Label']).transform(train_s['Label']))
#val_s_labs = np_utils.to_categorical(encoder.fit(val_s['Label']).transform(val_s['Label']))
#train_p_labs = np_utils.to_categorical(encoder.fit(train_p['Label']).transform(train_p['Label']))
#val_p_labs = np_utils.to_categorical(encoder.fit(val_p['Label']).transform(val_p['Label']))


#unique, counts = np.unique(val_p_labs, return_counts=True)

#print(np.asarray((unique, counts)).T)

[[0.00e+00 1.44e+03]
 [1.00e+00 2.88e+02]]


In [66]:
#1 hot encode labels
train_s_labs = pd.get_dummies(data = train_s, columns=['Label'])
val_s_labs = pd.get_dummies(data = val_s, columns=['Label'])
train_p_labs = pd.get_dummies(data = train_p, columns=['Label'])
val_p_labs = pd.get_dummies(data = val_p, columns=['Label'])

val_p_labs

,File Name,Path from Root,Label_antalgic,Label_lurching,Label_normal,Label_steppage,Label_stifflegged,Label_trendelenburg
974,Sbj3_steppage_T1_P,subject3/steppage/trial1/,0,0,0,1,0,0
2810,Sbj9_normal_T12_P,subject9/normal/trial12/,0,0,1,0,0,0
1336,Sbj8_lurching_T8_P,subject8/lurching/trial8/,0,1,0,0,0,0
1328,Sbj8_lurching_T6_P,subject8/lurching/trial6/,0,1,0,0,0,0
337,Sbj5_stiff-legged_T8_P,subject5/stiff-legged/trial8/,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
98,Sbj2_stiff-legged_T19_P,subject2/stiff-legged/trial19/,0,0,0,0,1,0
191,Sbj2_normal_T2_P,subject2/normal/trial2/,0,0,1,0,0,0
786,Sbj4_antalgic_T17_P,subject4/antalgic/trial17/,1,0,0,0,0,0
1613,Sbj6_normal_T15_P,subject6/normal/trial15/,0,0,1,0,0,0


Train/test split

## Preprocessing functions
Common useful functions

Skeleton specific functions

Plate specific functions

In [162]:
#file_name = 'Sbj4_antalgic_T17_P' #pressure example
file_name = 'Sbj2_steppage_T13_S.csv' #skeleton example
#a = reference_df[reference_df['File Name']==file_name,'Label']
a = reference_df.loc[reference_df['File Name']==file_name,'Type'] == 'skeleton'
a
#print(reference_df.loc[reference_df['File Name']==file_name,'Type'] == 'pressure')


4    True
Name: Type, dtype: bool

In [174]:
filepath = root_directory + '/dataset/' + reference_df.loc[reference_df['File Name']==file_name,'Path from Root'] + file_name
filepath.iloc[0]


#df_skelly = pd.read_csv(filepath.iloc[0], header=None)
#df_skelly = df_skelly.drop([df_skelly.columns[0],df_skelly.columns[-1]], axis=1)
#df_skelly


'/Users/lorenzobarbiero/Documents/GitHub/CNN-Gait-recognition/dataset/subject2/steppage/trial13/Sbj2_steppage_T13_S.csv'

## Input data pipeline

In [183]:
#load data function
def load_data(file_name, ref_df):
    #if isinstance(data_dir, bytes):
    #    data_dir = data_dir.decode()
    #if isinstance(file_name, bytes):
    #    file_name = file_name.decode()
        
    print(file_name)
    
    #skeleton data
    check = ref_df.loc[ref_df['File Name']==file_name,'Type'] == 'skeleton'
    
    if check.iloc[0] == True:
        # Specify the path to the CSV file
        filepath = root_directory + '/dataset/' + ref_df.loc[ref_df['File Name']==file_name,'Path from Root'] + file_name
        print(filepath.iloc[0])
    
        # Read the CSV file into a pandas DataFrame, Drop time signature column and last empty column
        df_skelly = pd.read_csv(filepath.iloc[0], header=None)
        df_skelly = df_skelly.drop([df_skelly.columns[0],df_skelly.columns[-1]], axis=1)
        
        #convert to np array and reshape
        data = df_skelly.to_numpy().reshape((-1,32,3))        
        print(data.shape)
    else:
        None
        
    return data.squeeze()

In [186]:
a = load_data(file_name, reference_df)

Sbj2_steppage_T13_S.csv
/Users/lorenzobarbiero/Documents/GitHub/CNN-Gait-recognition/dataset/subject2/steppage/trial13/Sbj2_steppage_T13_S.csv
(262, 32, 3)


(262, 32, 3)